# Load Variables

In [1]:
# Set environment variables

import os
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ["AZURE_OPENAI_ENDPOINT_SCUS"]
os.environ["AZURE_OPENAI_API_KEY"]  = os.environ["AZURE_OPENAI_API_KEY_SCUS"]
os.environ["OPENAI_API_VERSION"]    = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["AZURE_OPENAI_API_TYPE"] = os.environ["OPENAI_API_TYPE"]

MODEL = os.environ["GPT4-0125PREVIEW-128k"]

# https://smith.langchain.com/
os.environ["LANGCHAIN_TRACING_V2"]  = "true"
os.environ["LANGCHAIN_PROJECT"]     = "langgraph_04 Custom Agents"
os.environ["LANGCHAIN_API_KEY"]     = os.environ["LANGCHAIN_API_KEY"]

question_basic                      = "What's the date of Easter 2021?"
question_for_tools                  = "My name is Mauro. What do I get, if I apply the magic tool to the number of characters of my name?"
question_follow_up                  = "Is this result an even or odd number?"

## Automate a [Tool Calling Agent](https://python.langchain.com/docs/modules/agents/agent_types/tool_calling/) creation
Let's recall that this agent has the following three requirements:
1) One or more functions, each one associated to a tool, all included in a `tools` list
2) An LLM
3) A ChatPromptTemplate that includes a `MessagesPlaceholder("agent_scratchpad")`

Now we create 1) and 2) which are common to every agent.<br/>
For the implementation of 3), the only custom component is the system message so we create an helper function called `create_agent_executor()`

### Automate a Tool Calling Agent creation --> Create the tools this agent has access to
- Create one or more functions
- Decorate each function as a tool
- Create a list containing all tools defined

In [2]:
# Create some tools, then add them to the "tools" list

from langchain.tools import tool
@tool("internet_search_tool", return_direct=False) # this is the name tracked in LangSmith
def internet_search_function(query: str) -> str:
    """Searches the internet using DuckDuckGo."""
    from duckduckgo_search import DDGS
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=5)]
        return results if results else "No results found."

@tool("process_content_tool", return_direct=False)
def process_content_function(url: str) -> str:
    """Extracts the textual content from a webpage."""
    import requests
    from bs4 import BeautifulSoup
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

tools = [internet_search_function, process_content_function]


first_url = internet_search_function(question_basic)[0]['href']
content = process_content_function(first_url)

# print(content) # just for testing

### Automate a Tool Calling Agent creation --> Create the LLM

In [3]:
# Create LLM object
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=1000)

### Create the "Tool Calling Agent" --> Create the ChatPromptTemplate object through an helper function
**Important**: `Tool Calling Agent` requires an additional variable called `agent_scratchpad`. Intermediate agent actions and tool output messages will be passed in here.

In [4]:
# Helper function for creating agents
def create_agent_executor(llm: AzureChatOpenAI, tools: list, system_prompt: str):
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
    from langchain.agents import AgentExecutor
    from langchain.agents.openai_tools.base import create_openai_tools_agent
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    return agent_executor

## Create the **Search Agent Executor**

In [5]:
search_agent_exec = create_agent_executor(
    system_prompt="You are a web searcher. Search the internet for information.",
    llm=llm,
    tools=tools)

search_agent_exec

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a web sear

In [6]:
print(f"question_basic: {question_basic}")
search_agent_exec.invoke({"messages": [("human", question_basic)]})

question_basic: What's the date of Easter 2021?


> Entering new AgentExecutor chain...

Invoking: `internet_search_tool` with `{'query': 'date of Easter 2021'}`


[{'title': 'Easter 2021 - Calendar Date', 'href': 'https://www.calendardate.com/easter_2021.htm', 'body': '28. 29. 30. Easter for the year 2021 is celebrated/ observed on Sunday, April 4th. Easter also called Resurrection Sunday or Pascha is one of the most important days in the Christian faith commemorating the resurrection of Jesus Christ from the dead according to the New Testament.'}, {'title': 'Easter 2021 - Calendar-12.com', 'href': 'https://www.calendar-12.com/holidays/easter/2021', 'body': 'Easter 2021. Easter (Easter Sunday) or Pascha is the oldest and most important Christian feast, celebrating the Resurrection of Jesus Christ on the third day after his crucifixion, as described in the New Testament. Easter is preceded by Lent, a forty-day period of fasting and penance that starts on Ash Wednesday .'}, {'title': 'L

{'messages': [('human', "What's the date of Easter 2021?")],
 'output': 'Easter for the year 2021 was celebrated on Sunday, April 4th.'}

In [ ]:
print(f"question_basic: {question_basic}")
list(search_agent_exec.stream({"messages": [("human", question_basic)]}))

## Create the **Insights Research Agent Executor**

In [7]:
insights_research_agent_exec = create_agent_executor(
    system_prompt = """You are a Insight Researcher. Do step by step. 
        Based on the provided content first identify the list of topics,
        then search internet for each topic one by one
        and finally find insights for each topic one by one.
        Include the insights and sources in the final response""",
    llm = llm,
    tools = tools)

insights_research_agent_exec

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a Insight 

# NODES

## Helper function to build a node

In [8]:
# This function facilitates creating nodes in the graph:
# it takes care of invoking the agent executor and then converts its response to a human message. 
# That is how we will add it the global state of the graph

def agent_node(state, agent_executor, node_name):
    from langchain_core.messages import HumanMessage
    result = agent_executor.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=node_name)]}

## Node creation function for the "Web_Searcher" agent

In [9]:
# create the search node create function, starting from the search agent executor

import functools

search_node_create_function = functools.partial(agent_node, agent_executor=search_agent_exec, node_name="Web_Searcher")

In [10]:
search_node_create_function(state={"messages": [("human", "What's the weather like in Rome, Italy?")]})



> Entering new AgentExecutor chain...

Invoking: `internet_search_tool` with `{'query': 'current weather in Rome, Italy'}`


[{'title': 'Rome, Italy 14 day weather forecast - timeanddate.com', 'href': 'https://www.timeanddate.com/weather/italy/rome/ext', 'body': 'Rome 14 Day Extended Forecast. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Currently: 70 °F. Passing clouds. (Weather station: Rome Urbe Airport, Italy). See more current weather.'}, {'title': 'Rome, Lazio, Italy Current Weather | AccuWeather', 'href': 'https://www.accuweather.com/en/it/rome/213490/current-weather/213490', 'body': 'Current weather in Rome, Lazio, Italy. Check current conditions in Rome, Lazio, Italy with radar, hourly, and more.'}, {'title': 'Rome, Lazio, Italy Weather Forecast | AccuWeather', 'href': 'https://www.accuweather.com/en/it/rome/213490/weather-forecast/213490', 'body': 'Rome, Lazio, Italy Weather Forecast, with current conditions, wind, air quality, and 

{'messages': [HumanMessage(content="The current weather in Rome, Italy is as follows:\n\n- Temperature: 17°C with passing clouds.\n- Feels Like: 17°C\n- Forecast: High of 18°C and a low of 6°C\n- Wind: 15 km/h from the Northwest\n- Humidity: 45%\n- Dew Point: 5°C\n\nFor the upcoming hours, the temperature is expected to decrease gradually, reaching 11°C by 23:00.\n\nPlease note that weather conditions can change rapidly, so it's always a good idea to check for the most current information closer to your time of interest.", name='Web_Searcher')]}

## Node creation function for the "Insights_Researcher" agent

In [11]:
# create the search node create function, starting from the search agent executor

import functools

researcher_node_create_function = functools.partial(agent_node, agent_executor=insights_research_agent_exec, node_name="Insights_Researcher")

## Create the Supervisor Agent as a Custom Agent for Tools

### Build Tools

In [12]:
# given the two nodes...
members = ["Web_Searcher", "Insights_Researcher"]

# ...the potential actions identified by the supervisor include "FINISH"
options = ["FINISH"] + members

print(f"members: {members},\noptions: {options}")

members: ['Web_Searcher', 'Insights_Researcher'],
options: ['FINISH', 'Web_Searcher', 'Insights_Researcher']


In [13]:
# Using openai function calling can make output parsing easier for us

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

tools = [function_def]

tools

[{'name': 'route',
  'description': 'Select the next role.',
  'parameters': {'title': 'routeSchema',
   'type': 'object',
   'properties': {'next': {'title': 'Next',
     'anyOf': [{'enum': ['FINISH', 'Web_Searcher', 'Insights_Researcher']}]}},
   'required': ['next']}}]

### LLM "bound" to the tools above

In [14]:
# To pass in our tools to the agent, we just need to format them to the OpenAI tool format and pass them to our model. 
# By bind-ing the functions, we’re making sure that they’re passed in each time the model is invoked.

# llm_bound_to_tools = llm.bind_tools(tools=tools, tool_choice="route")
llm_bound_to_tools = llm.bind_functions(functions=tools, function_call="route")
llm_bound_to_tools

RunnableBinding(bound=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fea1a62a510>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fea1a706cd0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1000, azure_endpoint='https://mmopenaiscus.openai.azure.com/', deployment_name='gpt4-0125preview-128k', openai_api_version='2024-02-15-preview', openai_api_type='azure'), kwargs={'functions': [{'name': 'route', 'description': 'Select the next role.', 'parameters': {'title': 'routeSchema', 'type': 'object', 'properties': {'next': {'title': 'Next', 'anyOf': [{'enum': ['FINISH', 'Web_Searcher', 'Insights_Researcher']}]}}, 'required': ['next']}}], 'function_call': {'name': 'route'}})

### cpt_supervisor creation

This ChatPromptTemplate called `cpt_supervisor` would actually take three input variables: `members`, `messages` and `options`.<br/>
However, by using the `partial` function, we hard-code two variables (`members` and `options`), so that only `messages` is expected when invoking this template.<br/>
The `partial()` method in this case is used to create a version of the template with some variables pre-filled, which reduces the number of variables that need to be provided when the template is used later.<br/>
This effectively means that `members` and `options` do not need to be provided as input variables when using the second object, as they are already included as part of the template. More specifically, they are automatically passed with these values (both string types):
- `members` --> "'Web_Searcher', 'Insights_Researcher'"
- `options` --> "'FINISH', 'Web_Searcher', 'Insights_Researcher'"

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt_supervisor = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.")

mp1 = MessagesPlaceholder(variable_name="messages")

mp2 = (
    "user",
    "Given the conversation above, who should act next?"
    " Or should we FINISH? Select one of: {options}")

cpt_supervisor = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_supervisor),
        mp1,
        mp2
    ]
).partial(options=str(options), members=", ".join(members))

cpt_supervisor

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Web_Searcher', 'Insights_Researcher']", 'members': 'Web_Searcher, Insights_Researcher'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['members'], template='You are a supervisor tasked with managing a conversation between the following workers: {members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['option

In [16]:
# This is just to demonstrate that cpt_supervisor may be called just passing the "messages" variable

cpt_supervisor.invoke ({"messages": [("human", "Hi, I'm looking for this evening TV programs")]})

ChatPromptValue(messages=[SystemMessage(content='You are a supervisor tasked with managing a conversation between the following workers: Web_Searcher, Insights_Researcher. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.'), HumanMessage(content="Hi, I'm looking for this evening TV programs"), HumanMessage(content="Given the conversation above, who should act next? Or should we FINISH? Select one of: ['FINISH', 'Web_Searcher', 'Insights_Researcher']")])

## Create the **supervisor** agent leveraging the objects collected above

In [17]:
# We're ready to put together the three elements needed to create the node as a pipeline:
# - ChatPromptTemplate
# - RunnableBinding
# - OpenAIToolsAgentOutputParser
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

supervisor_agent = (
    cpt_supervisor
    | llm_bound_to_tools
    # | OpenAIToolsAgentOutputParser()
    | JsonOutputFunctionsParser()
)

supervisor_agent

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Web_Searcher', 'Insights_Researcher']", 'members': 'Web_Searcher, Insights_Researcher'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['members'], template='You are a supervisor tasked with managing a conversation between the following workers: {members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['option

In [18]:
tools

[{'name': 'route',
  'description': 'Select the next role.',
  'parameters': {'title': 'routeSchema',
   'type': 'object',
   'properties': {'next': {'title': 'Next',
     'anyOf': [{'enum': ['FINISH', 'Web_Searcher', 'Insights_Researcher']}]}},
   'required': ['next']}}]

In [19]:
# supervisor_agent.invoke ({"messages": [("human", "Hi, I'm looking for this evening TV programs")]})
list(supervisor_agent.stream ({"messages": [("human", "Hi, I'm looking for this evening TV programs")]}))

[{},
 {'next': ''},
 {'next': 'Web'},
 {'next': 'Web_Search'},
 {'next': 'Web_Searcher'}]

In [20]:
supervisor_agent.invoke ({"messages": [("human", "Hi, I need to think about how to write my next book")]})

{'next': 'Insights_Researcher'}

In [21]:
supervisor_agent.invoke ({"messages": [("human", "Hi, I'm done")]})

{'next': 'FINISH'}

In [ ]:
# supervisor_agent.invoke ({"messages": [("human", "Hi, I need look for the TV programs of tonight")]})
#list(supervisor_agent.stream ({"messages": [("human", "Hi, I need look for the TV programs of tonight")]}))

from langchain.agents import AgentExecutor
# supervisor_agent_executor = AgentExecutor(agent=supervisor_agent, tools=tools, verbose=True)

# doesn't work, but doesn't matter ;-)
# list(supervisor_agent_executor.stream({"input": {"messages": [("human", "Hi, I need look for the TV programs of tonight")]}}))

# GRAPH!

Now that we have Tools, Agents and Nodes we are ready to build the graph

In [22]:
# First, we need to define the "AgentState", which represents the conversation history
from langchain_core.messages import BaseMessage
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import operator

class AgentState(TypedDict):    
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

In [23]:
# Then, we create the graph using from langgraph.graph import StateGraph
from langgraph.graph import StateGraph

workflow = StateGraph(AgentState)

In [24]:
# To start building the workflow, we add the three nodes

workflow.add_node("Web_Searcher", search_node_create_function) # functools.partial
workflow.add_node("Insights_Researcher", researcher_node_create_function) # functools.partial
workflow.add_node("Supervisor", supervisor_agent) # langchain_core.runnables.base.RunnableSequence

[n for n in workflow.nodes]

['Web_Searcher', 'Insights_Researcher', 'Supervisor']

In [25]:
# Now we add the edges between the two node members and the supervisor
# We want our workers to ALWAYS "report back" to the Supervisor node when done

for member in members:
    print(f"connecting {member} node to the supervisor node...")
    workflow.add_edge(member, "Supervisor")
    
[e for e in workflow.edges]

connecting Web_Searcher node to the supervisor node...
connecting Insights_Researcher node to the supervisor node...


[('Insights_Researcher', 'Supervisor'), ('Web_Searcher', 'Supervisor')]

In [26]:
# The conditional map illustrates the match between each node and its direction
from langgraph.graph import StateGraph, END

conditional_map = {k: k for k in members}

conditional_map["FINISH"] = END

conditional_map

{'Web_Searcher': 'Web_Searcher',
 'Insights_Researcher': 'Insights_Researcher',
 'FINISH': '__end__'}

In [27]:
# Associate the conditional map to the workflow

workflow.add_conditional_edges(
    "Supervisor", 
    lambda x: x["next"], 
    conditional_map)

In [28]:
# Add entry point

workflow.set_entry_point("Supervisor")

In [29]:
# Compile the graph

graph = workflow.compile()

graph

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<messages,next>(writes=[ChannelWriteEntry(channel='messages', value=_get_state_key(), skip_none=False), ChannelWriteEntry(channel='next', value=_get_state_key(), skip_none=False)]), ChannelWrite<start:Supervisor>(writes=[ChannelWriteEntry(channel='start:Supervisor', value='__start__', skip_none=False)])]), 'Web_Searcher': PregelNode(config={'tags': []}, channels={'messages': 'messages', 'next': 'next'}, triggers=['branch:Supervisor:condition:Web_Searcher'], mapper=functools.partial(<function _coerce_state at 0x7fea12f90180>, <class '__main__.AgentState'>), writers=[ChannelWrite<Web_Searcher,messages,next>(writes=[ChannelWriteEntry(channel='Web_Searcher', value=None, skip_none=False), ChannelWriteEntry(channel='messages', value=_get_state_key(), skip_none=False), ChannelWriteEntry(channel='next', value=_get_state_key(), skip_none=F

In [30]:
# Run the graph

from langchain_core.messages import HumanMessage

for s in graph.stream({
    "messages": [HumanMessage(content="""Search for the latest AI technology trends in 2024,
            summarize the content. After summarise pass it on to insight researcher
            to provide insights for each topic""")]
}):
    if "__end__" not in s:
        print(s)
        print("----")

{'Supervisor': {'next': 'Web_Searcher'}}
----


> Entering new AgentExecutor chain...

Invoking: `internet_search_tool` with `{'query': 'latest AI technology trends in 2024'}`


[{'title': "What's next for AI in 2024 | MIT Technology Review", 'href': 'https://www.technologyreview.com/2024/01/04/1086046/whats-next-for-ai-in-2024/', 'body': 'In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI models ...'}, {'title': 'The most important AI trends in 2024 - IBM Blog', 'href': 'https://www.ibm.com/blog/artificial-intelligence-trends/', 'body': '2022 was the year that generative artificial intelligence (AI) exploded into the public consciousness, and 2023 was the year it began to take root in the business world. 2024 thus stands to be a pivotal year for the future of AI, as researchers and enterprises seek to establish how this evolutionary leap in technology c